# Demo: Using Portfolio Snapshots to run risk measure, net_delta, and portfolio valuation

This notebook demonstrates how to use Portfolio Snapshots to run risk measure, net_delta, and portfolio valuation APIs.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import pandas as pd

# create an alias to the api
api = connect_widget.get_api()

## List all of the available portfolio metadatas

In [ ]:
metadata_list = api.portfolio().list_portfolio_metadata()
metadata_list

## Get all snapshots associated with a metadata_id

In [ ]:
# pick which portfolio metadata that we want to retrieve its snapshots from
# for demo purpose, pick the first metadata with account (if any) from the available metadata
metadata_with_acc = [m for m in metadata_list if m.account_ids is not None]
metadata_id = metadata_with_acc[0].metadata_id if metadata_with_acc else metadata_list[0].metadata_id

# get all snapshots associated with it
all_snapshots = []
offset = 0
limit = 500
still_has_records = True
max_records = 1000  # for demo purpose, limit large portfolio records
while still_has_records and offset < max_records:
    snapshots = api.portfolio().list_portfolio_snapshots(metadata_id, offset=offset, limit=limit)
    all_snapshots.extend(snapshots)
    if len(snapshots) < limit:
        still_has_records = False
    else:
        offset = offset + limit

## (Optional) pick latest snapshot for each date (daily frequency portfolio)

In [ ]:
daily_snapshots = {}
for snapshot in all_snapshots:
    as_of_date = pd.Timestamp(snapshot.as_of_time).floor('D')
    if daily_snapshots.get(as_of_date) is None:
        daily_snapshots[as_of_date] = snapshot
daily_snapshots

## Run Risk Measures

In [ ]:
from serenity_types.risk import measures as risk_measures
from serenity_sdk.types.measures import RiskMeasureContext

# Risk measure parameters
measures = [
    risk_measures.MeasureParameters(
        tag="CVaR 99%",
        measure_type=risk_measures.MeasureType.CVAR,
        confidence_level="0.99",
    ),
    risk_measures.MeasureParameters(
        tag="VaR 99%",
        measure_type=risk_measures.MeasureType.VAR,
        confidence_level="0.99",
    ),
    risk_measures.MeasureParameters(
        tag="CVaR 95%",
        measure_type=risk_measures.MeasureType.CVAR,
        confidence_level="0.95",
    ),
    risk_measures.MeasureParameters(
        tag="Downside Dev",
        measure_type=risk_measures.MeasureType.DOWNDEV,
    ),
    risk_measures.MeasureParameters(
        tag="Standard Dev",
        measure_type=risk_measures.MeasureType.STDEV,
    ),
    risk_measures.MeasureParameters(
        tag="Upside Dev",
        measure_type=risk_measures.MeasureType.UPDEV,
    ),
    risk_measures.MeasureParameters(
        tag="CGaR 95%",
        measure_type=risk_measures.MeasureType.CGAR,
        confidence_level="0.95",
    ),
    risk_measures.MeasureParameters(
        tag="GaR 99%",
        measure_type=risk_measures.MeasureType.GAR,
        confidence_level="0.99",
    ),
    risk_measures.MeasureParameters(
        tag="CGaR 99%",
        measure_type=risk_measures.MeasureType.CGAR,
        confidence_level="0.99",
    ),
]
risk_computation_request = risk_measures.RiskComputationRequest(
    lookback_days=365, sampling_hours=1, horizon_scale="24.0", measures=measures
)

# run risk measures for each snapshots
pf_risk_measures_results = {}
for as_of_date, pf_snapshot in daily_snapshots.items():
    print(f'as_of_date: {as_of_date}')
    rm_ctx = RiskMeasureContext(request=risk_computation_request, as_of_time=as_of_date)
    legacy_portfolio = api.portfolio().to_legacy_portfolio(pf_snapshot)
    pf_risk_measures_results[as_of_date] = api.risk().compute_risk_measures(rm_ctx, legacy_portfolio)
pf_risk_measures_results

## Run Net-Delta

In [ ]:
from serenity_types.pricing.core import CashTreatment, PricingContext

asset_master = api.refdata().load_asset_master()
usd_asset_id = asset_master.get_asset_id_by_symbol('ccy.usd', 'SERENITY')

# run net_delta conversion for each snapshots
pf_net_delta_results = {}
for as_of_date, pf_snapshot in daily_snapshots.items():
    ctx = PricingContext(as_of_time=as_of_date, cash_treatment=CashTreatment.FIAT_PEGGED_STABLECOINS, base_currency_id=usd_asset_id)
    legacy_portfolio = api.portfolio().to_legacy_portfolio(pf_snapshot)
    pf_net_delta_results[as_of_date] = api.valuation().compute_portfolio_net_delta_value(ctx, legacy_portfolio)
pf_net_delta_results

## Run Portfolio Valuation from result of Net-Delta

In [ ]:
from serenity_sdk.types.common import Portfolio

# run portfolio valuation for each net delta-ed positions
pf_valuation_results = {}
for as_of_date, net_delta_pos in pf_net_delta_results.items():
    ctx = PricingContext(as_of_time=as_of_date, cash_treatment=CashTreatment.FIAT_PEGGED_STABLECOINS, base_currency_id=usd_asset_id)
    net_delta_pf = Portfolio({a.asset_id: a.quantity for a in net_delta_pos.net_delta_asset_positions})
    pf_valuation_results[as_of_date] = api.valuation().compute_portfolio_value(ctx, net_delta_pf)
pf_valuation_results